In [2]:
import newspaper
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import sqlite3
import pandas as pd
import time
from datetime import datetime

In [18]:
def scrape_cnn_links(driver):
    els = driver.find_elements_by_class_name('cnn-search__results-list')
    links = []
    for article in els[0].find_elements_by_class_name('cnn-search__result--article'):
        link = article.find_elements_by_class_name('cnn-search__result-contents')[0].\
                find_elements_by_class_name('cnn-search__result-headline')[0].\
                find_elements_by_tag_name('a')[0].get_attribute('href')

        links.append(link)
    return links

In [19]:
def update_articles_table(links, site):
    bad_links = []
    with sqlite3.connect('data/newspaper.db') as conn:
        c = conn.cursor()
        for link in links:
            c.execute("""SELECT link
                        FROM articles
                        WHERE link="{}"
                        """.format(link))
            
            result = c.fetchone()
            if result:
                print("link already exists: {}".format(link))
                bad_links.append(link)
            else:
                c.execute("INSERT INTO articles VALUES (?, ?)", (site, link))
                conn.commit()
        return bad_links 

In [22]:
driver = webdriver.Firefox()

In [23]:
for i in range(0, 15):
    print(i)
    
    if i == 0:
        url = "https://www.cnn.com/search?size=100&q=coronavirus&sort=newest"
    else:
        url = "https://www.cnn.com/search?size=100&q=coronavirus&sort=newest&from={}&page={}".format(100*i, i+1)
    
    driver.get(url)
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print('sleeping... {}'.format(current_time))
    time.sleep(60)
    
    links = scrape_cnn_links(driver)
    bad_links = update_articles_table(links, 'cnn')
    if len(bad_links) == len(links):
        break

0
sleeping... 10:30:46
link already exists: https://www.cnn.com/asia/live-news/coronavirus-outbreak-03-05-20-intl-hnk/index.html
link already exists: https://www.cnn.com/2020/03/03/health/us-coronavirus-cases-state-by-state/index.html
link already exists: https://www.cnn.com/2020/03/04/health/new-york-coronavirus-cases/index.html
link already exists: https://www.cnn.com/2020/03/04/business/starbucks-coronavirus/index.html
link already exists: https://www.cnn.com/2020/03/04/football/liverpool-chelsea-fa-cup-jurgen-klopp-spt-intl/index.html
link already exists: https://www.cnn.com/2020/01/29/asia/coronavirus-cases-worldwide-intl-hnk/index.html
link already exists: https://www.cnn.com/2020/03/02/asia/pets-coronavirus-spread-intl-hnk/index.html
link already exists: https://www.cnn.com/2020/03/04/health/coronavirus-test-what-happens-explainer/index.html
link already exists: https://www.cnn.com/business/live-news/stock-market-news-today-030420/index.html
link already exists: https://www.cnn.

In [126]:
article = newspaper.Article(links[1])
article.download()
article.parse()

In [128]:
article.text

'(CNN) The first novel coronavirus cases were reported in Florida and New York on Sunday while officials in Washington state confirmed the second US death from the outbreak.\n\nThe New York patient is a woman in her late 30s, who contracted the virus while traveling in Iran and is currently isolated in her home, New York Gov. Andrew Cuomo said.\n\nWe have learned of the 1st positive case of COVID-19 in NY. The patient contracted the virus while in Iran & is isolated There is no reason for undue anxiety—the general risk remains low in NY. We are diligently managing this situation &will provide info as it becomes available. pic.twitter.com/rLnObvrg3R\n\n"The patient has respiratory symptoms, but is not in serious condition and has been in a controlled situation since arriving to New York," Cuomo\'s statement read.\n\nThe New York State Department of Health said the patient went to a hospital in Manhattan after contracting the virus.\n\nCuomo said approval for state testing was granted Sa

In [130]:
links[1]

'https://www.cnn.com/2020/03/01/health/us-coronavirus-sunday/index.html'